In [1]:
# check langchain version it should be 0.3 if not uncomment below pip command to install the correct version
import langchain, langchain_community
print(langchain.__version__)
print(langchain_community.__version__)
# !pip install langchain==0.3.27 langchain-openai==0.3.33 langchain-community==0.3.24

0.3.27
0.3.24


In [2]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.tools import tool
# Load environment variables
load_dotenv()

import warnings
warnings.filterwarnings('ignore')
# The LLM 
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

Let’s create a Goal-Oriented Agent for the doctor appointment scenario.

Unlike the task-oriented agent, this one will:

Take a final goal (“Book a dermatology appointment tomorrow afternoon”)

Decide all the necessary steps automatically:

Find doctors

Check availability

Book the appointment

Return a final confirmation in one go

We’ll still use dynamic tools like before, but the agent now plans multiple steps toward a goal.

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools import tool

In [10]:
# ----------------------
# Simulated Data (same as task-oriented)
# ----------------------
DOCTORS_DB = {
    "Dermatology": ["Dr. Smith", "Dr. Kim"],
    "Cardiology": ["Dr. Patel", "Dr. Lee"],
    "Neurology": ["Dr. Jones", "Dr. Wang"]
}

APPOINTMENT_SLOTS = {
    "Dr. Smith": ["Tomorrow 10AM", "Tomorrow 3PM"],
    "Dr. Kim": ["Tomorrow 11AM", "Tomorrow 4PM"],
    "Dr. Patel": ["Jan 21 9AM", "Jan 22 2PM"],
    "Dr. Lee": ["Jan 20 10AM", "Jan 22 1PM"],
    "Dr. Jones": ["Jan 23 11AM", "Jan 24 3PM"],
    "Dr. Wang": ["Jan 21 9AM", "Jan 24 2PM"]
}

In [4]:
# Tools

@tool
def FindDoctorsTool(specialty: str):
    """Return available doctors for the given specialty."""
    doctors = DOCTORS_DB.get(specialty, [])
    if doctors:
        return f"Available {specialty} doctors: {', '.join(doctors)}"
    return f"No doctors found for specialty: {specialty}"

@tool
def CheckAvailabilityTool(doctor_name: str):
    """Return available appointment slots for the selected doctor."""
    slots = APPOINTMENT_SLOTS.get(doctor_name, [])
    if slots:
        return f"Available slots for Dr. {doctor_name}: {', '.join(slots)}"
    return f"No available slots found for Dr. {doctor_name}"

@tool
def BookAppointmentTool(details: str):
    """Book an appointment and return confirmation."""
    return f"✅ Appointment successfully booked for: {details}"

tools = [FindDoctorsTool, CheckAvailabilityTool, BookAppointmentTool]

In [7]:
# Goal-Oriented Prompt

prompt = ChatPromptTemplate.from_template("""
You are a **goal-oriented healthcare assistant AI**.

User will provide a **goal**, like:
"Book a dermatology appointment tomorrow afternoon".

Your job is to **plan all the steps required** to achieve the goal:
- Find doctors
- Check availability
- Book the appointment

Use reasoning format:
Thought:
Action: <tool-name>
Action Input: <arguments>

Repeat until goal is achieved.

Once complete, give **Final Answer** with:
- Doctor chosen
- Appointment slot
- Confirmation message

User Goal: {input}
{agent_scratchpad}
""")


In [8]:
agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,       # Show reasoning steps
    max_iterations=10   # Allow multiple steps to achieve goal
)

In [12]:

# Run Goal-Oriented Task

response = executor.invoke({"input": "Book a Neurology appointment tomorrow afternoon"})
print("\n--- Goal-Oriented Agent Output ---")
print(response["output"])




> Entering new AgentExecutor chain...

Invoking: `FindDoctorsTool` with `{'specialty': 'neurology'}`
responded: Thought: I need to find doctors who specialize in neurology.  
Action: functions.FindDoctorsTool  
Action Input: { specialty: "neurology" }  

---

Thought: After finding the doctors, I will check their availability for tomorrow afternoon.  
Action: functions.CheckAvailabilityTool  
Action Input: { doctor_name: "<chosen_doctor_name>" }  

---

Thought: Once I have the available appointment slots, I will book an appointment for the user.  
Action: functions.BookAppointmentTool  
Action Input: { details: "<appointment_details>" }  

---

Now, I will execute the first action to find doctors specializing in neurology.

No doctors found for specialty: neurology
Invoking: `FindDoctorsTool` with `{'specialty': 'neurology'}`


No doctors found for specialty: neurologyThought: I need to find doctors who specialize in neurology.  
Action: functions.FindDoctorsTool  
Action Input: { s

In production scenario: In a real-world healthcare system, this agent would interact with live systems, and several additional considerations apply:

| Component             | Currently                  | Production Version                                                                  |
| --------------------- | --------------------------- | ----------------------------------------------------------------------------------- |
| Doctor Database       | Hardcoded dictionary        | Connects to live hospital/clinic database (EMR/EHR system)                          |
| Appointment Slots     | Static slots in code        | Pulled in **real-time** from scheduling system/API                                  |
| Booking Tool          | Returns static confirmation | Calls hospital API to **actually book** appointment and handle conflicts            |
| Error Handling        | None                        | Handle scheduling conflicts, unavailable slots, API failures, retries               |
| Security & Compliance | None                        | Must enforce **HIPAA/GDPR compliance**, encrypt sensitive data, user authentication |
| Logging & Auditing    | Verbose prints              | Centralized logging for monitoring, auditing, and compliance                        |
| Autonomy              | Controlled in code          | Could integrate **human oversight** or approval for critical tasks                  |
| Scalability           | Single-user simulation      | Multi-user, multi-agent orchestration for enterprise systems                        |


Key Takeaways for Students

Goal-Oriented Agents are designed to achieve final objectives autonomously.

They plan multiple steps without user guidance for each intermediate step.

In production, they interact with real databases, APIs, and systems, and require robust:
Error handling
Security and compliance
Logging and monitoring
Scalability considerations

Contrasted with task-oriented agents, goal-oriented agents can complete workflows end-to-end, improving efficiency in enterprise use cases like healthcare scheduling, logistics, and personal assistants.